In [1]:
import os, shutil
from glob import glob

import numpy as np
import xarray as xr
from rasterio.warp import Resampling

from dask.distributed import LocalCluster, Client

In [9]:
proj_string = xr.open_dataset("../data/mnt/forsite2/tas/tas_Austria_2020_250m_v2.nc").lambert_conformal_conic.attrs["spatial_ref"]

In [10]:
args = [
    ("forsite2/pr/pr_Austria_{year}_250m_v2.1.nc", "pr", "precipitation"),
    ("forsite2/tasmin/tasmin_new_Austria_{year}_250m_v2.1.nc", "tasmin", "min_air_temp"),
    ("forsite2/tasmax/tasmax_new_Austria_{year}_250m_v2.1.nc", "tasmax", "max_air_temp"),
    ("forsite2/tas/tas_Austria_{year}_250m_v2.nc", "tas", "air_temperature"),
    ("forsite2/PET/PET_Austria_{year}_250m_v2.nc", "PET", "pot_evapotransp"),
    ("forsite2/wspd/wspd_{year}_250m.nc", "wspd", "wind_speed"),
    ("forsite2/hurs/hurs_Austria_{year}_250m_v2.nc", "hurs", "rel_humidity"),
    ("forsite2/tds/tds_{year}_250m.nc", "tds", "dew_point"),
    ("APOLIS/APOLIS-SHORT-DAILY_GLO_hori_daysum_kWh_{year}*.nc", "GLO_hori_daysum_kWh", "horizontal_global_radiation"),
]

In [ ]:
client = Client(LocalCluster(n_workers=8, memory_limit="3G", threads_per_worker=2))
client

In [ ]:
for year in range(2019, 2024):

    print(f"Working on year {year}...")

    for path, old_var_name, new_var_name in args:
        if (
            (year in [2019, 2022] and old_var_name not in ["pr", "tasmin", "tasmax"])
            or len(glob("*", root_dir=f"../data/input/{year}.zarr/{new_var_name}")) == 1792
        ):
            continue
        print("\t...", new_var_name)
        if old_var_name == "GLO_hori_daysum_kWh":
            ds = xr.open_zarr(f"../data/input/{year}.zarr", decode_coords="all")
            if os.path.isdir("tmp.zarr"):
                shutil.rmtree("tmp.zarr")
            for file in glob(os.path.join("../data/mnt", path.format(year=year))):
                tmp = xr.open_mfdataset(file, parallel=True)
                tmp = tmp.rio.write_crs(tmp.lambert_conformal_conic.attrs["crs_wkt"])[old_var_name].rename(new_var_name)
                tmp = (tmp.reset_coords(["lat", "lon"], drop=True)
                          .rio.reproject_match(ds, resampling=Resampling.average)
                          .reset_coords(["lambert_conformal_conic"], drop=True))
                if os.path.isdir("tmp.zarr/time"):
                    tmp.to_zarr("tmp.zarr", mode="a-", append_dim="time")
                else:
                    tmp.to_zarr("tmp.zarr", mode="w")
            with xr.open_zarr("tmp.zarr") as ds:
                delayed = ds.sortby("time").drop_encoding().chunk(chunks={"time": len(ds.time), "x": 41, "y": 37}).to_zarr(f"../data/input/{year}.zarr", mode="a-", compute=False)
        else:
            delayed = (xr.open_mfdataset(os.path.join("../data/mnt", path.format(year=year)), parallel=True)[old_var_name]
                        .rename(new_var_name).sortby("time").chunk(chunks={"time": -1, "x": 41, "y": 37}).rio.write_crs(proj_string)
                        .to_zarr(f"../data/input/{year}.zarr", mode="a-", compute=False))
        delayed.compute()
        if os.path.isdir("tmp.zarr"):
            shutil.rmtree("tmp.zarr")
    
    ds = xr.open_zarr(f"../data/input/{year}.zarr", decode_coords="all")
    if year not in [2019, 2022]:
        da_list = []
        for new_var_name in ["min_rel_hum", "max_rel_hum"]:
            if len(glob("*", root_dir=f"../data/input/{year}.zarr/{new_var_name}")) == 1792:
                continue
            print("\t...", new_var_name, "(graphs only)")
            minmax_inverted = "min" if new_var_name.startswith("max") else "max"
            beta = 17.625
            lamb_ = 243.04
            da_list.append((np.exp(beta*ds["dew_point"]/(lamb_+ds["dew_point"])-beta*ds[f"{minmax_inverted}_air_temp"]/(lamb_+ds[f"{minmax_inverted}_air_temp"]))).rename(new_var_name))
        delayed = xr.merge(da_list).drop_encoding().chunk(chunks={"time": len(ds.time), "x": 41, "y": 37}).to_zarr(f"../data/input/{year}.zarr", mode="a-", compute=False)
        print("\t... compute")
        delayed.compute()

    print("...done.")